In [1]:
!pip install transformers==4.19.2
!pip install rouge_metric
!pip install nltk==3.6.5
!pip install sentencepiece
from google.colab import drive
drive.mount('./mydata')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at ./mydata; to attempt to forcibly remount, call drive.mount("./mydata", force_remount=True).


In [2]:
import sys
sys.path.append('./mydata/MyDrive/CSNLP_Project/Bert_model_COQA')
sys.path.append('./mydata/MyDrive/CSNLP_Project/T5_model_COQAR')
sys.path.append('./mydata/MyDrive/CSNLP_Project/T5_model_COQAR/rewriting')

In [3]:
import collections
import glob
import os
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from transformers import (AdamW, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup, BertTokenizer, BertModel, BertConfig)
from processors.coqa import Extract_Features, Processor, Result
from processors.evaluate import CoQAEvaluator, parse_args
from processors.Bert_model import BertBaseUncasedModel, BertBaseUncasedModel_with_T5, load_dataset, Write_predictions, Write_predictions_with_T5

#from transformers import BertModel, BertPreTrainedModel

import csv
import numpy as np

import evaluation
import argparse
import qrdatasets
import models
from utils import *
import random
import t5small
import t5base
import nltk

import config
import json

# CoQA dataset file
train_file="coqa-train-v1.0.json"
predict_file="coqa-dev-v1.0.json"
cur_path = os.getcwd()
output_directory = cur_path + "/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models"
input_dir = cur_path + "/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data"
# can use either BERT base or BERT large
pretrained_model="bert-base-uncased"
# pretrained_model="bert-large-uncased"
# it's better to fine-tune Bert-base for 4 epoches than only one
epochs = 2
evaluation_batch_size=1
train_batch_size=2

### create now dataset (json file)

In [ ]:
### Load two fine-tuned models, but with diffetent load methods

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load bert
# bert_path = './mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_from_original_Surya_epoch4'
# bert_model = BertBaseUncasedModel.from_pretrained(bert_path) 
# bert_tokenizer = BertTokenizer.from_pretrained(bert_path, do_lower_case=True)
# bert_model.to(device)

# load t5
# t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_small_with_story_batch16_hist_3_mixed/epoch10'
t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_base_with_story_batch4_hist_20/epoch6.zip'

t5_model = torch.load(t5_path)
# t5_input_tokenizer = t5small.get_input_tokenizer()
# t5_output_tokenizer = t5small.get_output_tokenizer()
t5_input_tokenizer = t5base.get_input_tokenizer()
t5_output_tokenizer = t5base.get_output_tokenizer()
t5_model.to(device)

pass

In [4]:
# default parameters for T5
hparams = {
    'epochs' : 2,
    'learning_rate' : 0.00005,
    'batch_size' : 4,
    'weight_decay' : 0.0,
    'history_size' : 3,
    'dropout_rate' : 0.1,
    'include_story' : True,
    'model_size' : 'small'
}
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def make_t5_dataset(input_dir,file_type='train',t5_type='base',train_file=train_file):
    ### open the COQA dataset
    if file_type == 'train':
        # training dataset
        with open(os.path.join(input_dir, train_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    elif file_type == 'dev':
        # dev dataset
        with open(os.path.join(input_dir, predict_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    else:
        raise Exception("must specify a file type: train/dev")
        pass
    
    # create empty dictionary
    t5_data = {'input':[], 'references':[], 'context':[]}
    # create dataset for T5
    for input_d in input_data["data"]:
        questions = []
        assert len(input_d['questions']) == len(input_d['answers'])
        for id_q in range(len(input_d['questions'])):
            curr_question = input_d['questions'][id_q]['input_text']
            if '?' not in curr_question:
                if len(curr_question) == 0:
                    pass
                # some question in statement form end with '.'
                elif curr_question[-1] in ['.']:
                    if curr_question[-2:] == '/.':
                        curr_question = curr_question[:-2] + '?'
                    elif ('True or False' in curr_question) or ('Name' in curr_question):
                        pass
                    else:
                        # not worth to deal with such small portion of data
                        pass
                elif curr_question[-1] in ['/']:
                    curr_question = curr_question[:-1] + '?'
                else:
                    curr_question += '?'
            if id_q == 0:
                questions.append(curr_question)
            else:
                questions.append(input_d['answers'][id_q-1]['input_text'])
                questions.append(curr_question)
            t5_data['input'].append(questions.copy())
            t5_data['references'].append([''])
            t5_data['context'].append(input_d['story'])

    # T5 make dataset
    if t5_type == 'base':
        dataset = t5base.make_dataset(t5_data, hparams, cuda = True)
    elif t5_type == 'small':
        dataset = t5base.make_dataset(t5_data, hparams, cuda = True)
    else:
        raise Exception("must specify a t5 model type: base/small")
    
    return dataset

In [ ]:
# get rewritten answers from T5

def generate_new_dataset(input_dir,file_type='train',t5_type='base',t5_model=t5_model,method='append'):
    ### open the COQA dataset
    if file_type == 'train':
        # training dataset
        with open(os.path.join(input_dir, train_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    elif file_type == 'dev':
        # dev dataset
        with open(os.path.join(input_dir, predict_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    else:
        raise Exception("must specify a file type: train/dev")
        pass
        
    if 'repeat' not in method:
        # get dataset
        dataset = make_t5_dataset(input_dir,file_type=file_type,t5_type=t5_type,train_file=train_file)

        ### generate rewritten questions
        loader = DataLoader(dataset=dataset, batch_size=hparams['batch_size'])
        t5_model.cuda()
        t5_model.train(False)
        rewritten_qs = []
        for dic in tqdm(loader, desc="Generating rewritten questions"):
            output = t5_model.generate(input_ids = dic['input_ids'], attention_mask = dic['attention_mask'])
            pred = t5_output_tokenizer.batch_decode(output, skip_special_tokens = True)
            rewritten_qs += pred

    ### Create new training/dev dataset json file for Bert
    # append or replace with the generated question 
    count = 0
    for story_id in range(len(input_data["data"])):
        for question_id in range(len(input_data["data"][story_id]['questions'])):
            # if the rewritten question from T5 is failed, we use the original question.
            # also clean the question data
            curr_question = input_data["data"][story_id]['questions'][question_id]['input_text']
            if '?' not in curr_question:
                if len(curr_question) == 0:
                    pass
                # some question in statement form end with '.'
                elif curr_question[-1] in ['.']:
                    if curr_question[-2:] == '/.':
                        curr_question = curr_question[:-2] + '?'
                    elif ('True or False' in curr_question) or ('Name' in curr_question):
                        pass
                    else:
                        # not worth to deal with such small portion of data
                        pass
                elif curr_question[-1] in ['/']:
                    curr_question = curr_question[:-1] + '?'
                else:
                    curr_question += '?'

            if 'repeat' not in method:
                if len(rewritten_qs[count]) < 2:
                    pass
                # for failed rewritten questions(those not even questions)
                elif rewritten_qs[count][-1] != '?':
                    rewritten_qs[count] = curr_question
            if 'append' in method:
                # append the rewritten question
                input_data["data"][story_id]['questions'][question_id]['input_text'] = curr_question + (' '+rewritten_qs[count])
            elif 'replace' in method:
                # replace with the rewritten question
                input_data["data"][story_id]['questions'][question_id]['input_text'] = rewritten_qs[count]
            elif 'repeat' in method:
                # repeat the curent question twice for comparison
                input_data["data"][story_id]['questions'][question_id]['input_text'] = curr_question + (' '+curr_question)
            else:
                raise Exception("must specify a method: append/replace")
                pass
            count += 1

    # save as new dataset json file
    file_name = 'coqa-{}-v1.0-{}_with_T5.json'.format(file_type,method)
    # with open(os.path.join( input_dir, 'coqa-dev-v1.0-with_T5.json'), 'w', encoding="utf-8") as outfile:
    with open(os.path.join(input_dir, file_name), 'w', encoding="utf-8") as outfile:
        json.dump(input_data, outfile)

generate_new_dataset(input_dir,file_type='dev',t5_type='base',t5_model=t5_model,method='append_v3')

### train bert with t5 embedding

In [6]:
def train_bert_with_t5(bert_model, t5_model, bert_tokenizer, bert_dataset, t5_dataset, device, pretrained_model, t5_pooled=False, epochs_trained=0, start_iter=0, start_total_loss=0, batch_size=2, save_criteria=None):
    save_flag = True

    bert_train_sampler = SequentialSampler(bert_dataset) 
    bert_dataloader = DataLoader(bert_dataset, sampler=bert_train_sampler, batch_size=batch_size)
    t5_dataloader = DataLoader(dataset=t5_dataset, batch_size=batch_size)
    t_total = len(bert_dataloader) // 1 * epochs

    # Preparing optimizer and scheduler
    optimizer_parameters = [{"params": [p for n, p in bert_model.named_parameters() if not any(nd in n for nd in ["bias", "LayerNorm.weight"])],"weight_decay": 0.01,},{"params": [p for n, p in bert_model.named_parameters() if any(nd in n for nd in ["bias", "LayerNorm.weight"])], "weight_decay": 0.0}]
    optimizer = AdamW(optimizer_parameters,lr=1e-5, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=2000, num_training_steps=t_total)

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(pretrained_model, "optimizer.pt")) and os.path.isfile(os.path.join(pretrained_model, "scheduler.pt")):
        optimizer.load_state_dict(torch.load(
            os.path.join(pretrained_model, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(
            os.path.join(pretrained_model, "scheduler.pt")))

    counter = 1
    # epochs_trained = 0
    train_loss, loss = start_total_loss, 0.0
    bert_model.zero_grad()
    iterator = trange(0, int(epochs), desc="Epoch", disable=False)
    for epoch_id in iterator:
        epoch_iterator = tqdm(zip(bert_dataloader,t5_dataloader), desc="Iteration", disable=True)
        for i,batch in enumerate(epoch_iterator):
            # skip the first few iterations
            if (epoch_id < epochs_trained) or ((epoch_id == epochs_trained) and (i<=start_iter)):
                counter += 1
                continue
            batch_bert, batch_t5 = batch
            # model.encoder(input_ids=s, attention_mask=attn, return_dict=True)
            # pooled_sentence = output.last_hidden_state # shape is [batch_size, seq_len, hidden_size]
            # # pooled_sentence will represent the embeddings for each word in the sentence
            # # you need to sum/average the pooled_sentence
            # pooled_sentence = torch.mean(pooled_sentence, dim=1)
            batch_t5['input_ids'] = batch_t5['input_ids'].to(device)
            batch_t5['attention_mask'] = batch_t5['attention_mask'].to(device)
            t5_embdeding = t5_model.encoder(input_ids = batch_t5['input_ids'], attention_mask = batch_t5['attention_mask'], return_dict=False)
            bert_model.train()
            batch_bert = tuple(t.to(device) for t in batch_bert)
            # inputs = { "input_ids": batch_bert[0],"token_type_ids": batch_bert[1], "attention_mask": batch_bert[2],"start_positions": batch_bert[3],"end_positions": batch_bert[4],"rational_mask": batch_bert[5],"cls_idx": batch_bert[6]}
            # loss = bert_model(**inputs)
            loss = bert_model(batch_bert[0],t5_embdeding,batch_t5,t5_pooled,token_type_ids=batch_bert[1],attention_mask=batch_bert[2],\
                              start_positions=batch_bert[3],end_positions=batch_bert[4],rational_mask=batch_bert[5],cls_idx=batch_bert[6],head_mask=None)
            loss.backward()
            train_loss += loss.item()

            #   optimizing training parameters
            if (i + 1) % 1 == 0:
                optimizer.step()
                scheduler.step()  
                bert_model.zero_grad()
                counter += 1
                #   Saving model weights every 1000 iterations
                if counter % 1000 == 0:
                    output_dir = os.path.join(output_directory, "model_weights"+str(epochs_trained))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = bert_model.module if hasattr(bert_model, "module") else bert_model
                    model_to_save.save_pretrained(output_dir)
                    bert_tokenizer.save_pretrained(output_dir)
                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
            if (i+1) % 1000 == 0:
                print('iter: {}, loss: {}'.format(i,train_loss/counter))

            if save_criteria and save_flag and ((train_loss/counter)<save_criteria):
                save_flag = False
                variant_name = 'Bert_with_T5_embdding_cond'

                #   create output directory for model parameters and to write predictions
                if not os.path.exists(output_directory+'/'+variant_name) :
                    os.makedirs(output_directory+'/'+variant_name)
                            
                model_to_save = bert_model.module if hasattr(bert_model, "module") else bert_model
                model_to_save.save_pretrained(output_directory+'/'+variant_name)
                bert_tokenizer.save_pretrained(output_directory+'/'+variant_name)
                return train_loss/counter

    return train_loss/counter

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

method = 'append'
epochs = 4
start_iter = 0
epochs_trained = 3
start_total_loss = 0  #1.747721705783295 * (56999*2)
save_criteria = None
t5_pooled = True

pretrained_model = './mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_poled_embedding_epoch2_batch2/'
# pretrained_model="bert-base-uncased"

## load raw model
# t5 
t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_base_with_story_batch4_hist_20/epoch6.zip'
t5_model = torch.load(t5_path)
t5_input_tokenizer = t5base.get_input_tokenizer()   # or t5small 
t5_output_tokenizer = t5base.get_output_tokenizer()
t5_model.to(device)
t5_model.train(False)
# bert
config = BertConfig.from_pretrained(pretrained_model, return_dict=False)
bert_tokenizer = BertTokenizer.from_pretrained(pretrained_model)
bert_model = BertBaseUncasedModel_with_T5.from_pretrained(pretrained_model, from_tf=bool(".ckpt" in pretrained_model), config=config,cache_dir=None,)
bert_model.to(device)

## get datasets
t5_dataset = make_t5_dataset(input_dir,file_type='train',t5_type='base',train_file=train_file)
# cache_file_name = 'bert-base-uncased_train_with_T5_{}_v3.2'.format(method)
# train_file_name = 'coqa-train-v1.0-{}_v3_with_T5.json'.format(method)
cache_file_name = 'bert-base-uncased_train'
# cache_file_name = 'bert-base-uncased_train_with_T5_append_v3_all'
train_file_name = 'coqa-train-v1.0.json'

if cache_file_name is not None:
    cache_file = os.path.join(input_dir,cache_file_name)

if os.path.exists(cache_file):
    features_and_dataset = torch.load(cache_file)
    features, bert_dataset, examples = (
        features_and_dataset["features"],features_and_dataset["dataset"],features_and_dataset["examples"])
else:
    # train_file_name=None, use the original dataset
    bert_dataset = load_dataset(bert_tokenizer, input_dir=input_dir, evaluate=False, cache_file_name=cache_file_name, train_file_name=None, append_method='append')
    features_and_dataset = torch.load(cache_file)
    features, bert_dataset, examples = (
        features_and_dataset["features"],features_and_dataset["dataset"],features_and_dataset["examples"])

t5_dataset_new = []
for feature in features:
    t5_dataset_new.append(t5_dataset[feature.example_index])
t5_dataset = t5_dataset_new
del t5_dataset_new
del examples
del features
del features_and_dataset

# train
train_bert_with_t5(bert_model, t5_model, bert_tokenizer, bert_dataset, t5_dataset, device, pretrained_model, t5_pooled, epochs_trained=epochs_trained, \
                   start_iter=start_iter, start_total_loss=start_total_loss, batch_size=2, save_criteria=save_criteria)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertBaseUncasedModel_with_T5: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertBaseUncasedModel_with_T5 from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertBaseUncasedModel_with_T5 from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertBaseUncasedModel_with_T5 were not initialized from the model checkpoint at ber

iter: 999, loss: 5.486859989938858
iter: 1999, loss: 4.753607458305322
iter: 2999, loss: 4.201126125379345
iter: 3999, loss: 3.8192279854704108
iter: 4999, loss: 3.5686778184470835
iter: 5999, loss: 3.372409662055415
iter: 6999, loss: 3.2319617550937254
iter: 7999, loss: 3.119186821406317
iter: 8999, loss: 3.021361992759289
iter: 9999, loss: 2.947090112462539
iter: 10999, loss: 2.874695964008614
iter: 11999, loss: 2.8178673969777734
iter: 12999, loss: 2.75632369275295
iter: 13999, loss: 2.711875973024277
iter: 14999, loss: 2.6703872270563798
iter: 15999, loss: 2.635548570434484
iter: 16999, loss: 2.5998644132881914
iter: 17999, loss: 2.570510077396671
iter: 18999, loss: 2.53962521626975
iter: 19999, loss: 2.5179892607768486
iter: 20999, loss: 2.495632609299334
iter: 21999, loss: 2.4744654040295595
iter: 22999, loss: 2.454104571533488
iter: 23999, loss: 2.4388994706166804
iter: 24999, loss: 2.4219676967941868
iter: 25999, loss: 2.404442399628862
iter: 26999, loss: 2.385971111524858
iter

Epoch:  50%|█████     | 1/2 [7:24:12<7:24:12, 26652.50s/it]

iter: 999, loss: 2.0866245107399295
iter: 1999, loss: 2.0801539724235205
iter: 2999, loss: 2.074297689679974
iter: 3999, loss: 2.065809209649494
iter: 4999, loss: 2.058095580037011
iter: 5999, loss: 2.049800467169048
iter: 6999, loss: 2.041416174390211
iter: 7999, loss: 2.0337118963546574
iter: 8999, loss: 2.025173434316342
iter: 9999, loss: 2.01749550983261
iter: 10999, loss: 2.0089170938594307
iter: 11999, loss: 2.00121487435743
iter: 12999, loss: 1.9912737813517964
iter: 13999, loss: 1.9827952360456673
iter: 14999, loss: 1.9748220115193666
iter: 15999, loss: 1.967216445131775
iter: 16999, loss: 1.9595122650745371
iter: 17999, loss: 1.9524295410340062
iter: 18999, loss: 1.9448329111133118
iter: 19999, loss: 1.9386545601324068
iter: 20999, loss: 1.9320863927424359
iter: 21999, loss: 1.9259368697209807
iter: 22999, loss: 1.9194505496086267
iter: 23999, loss: 1.9139686889627456
iter: 24999, loss: 1.908081156859474
iter: 25999, loss: 1.9018434391766925
iter: 26999, loss: 1.89505746485978

Epoch: 100%|██████████| 2/2 [14:49:02<00:00, 26671.27s/it]


1.747721705783295

In [ ]:
variant_name = 'Bert_with_T5_rewritten_epoch4_append_v3.2'

#   create output directory for model parameters and to write predictions
if not os.path.exists(output_directory+'/'+variant_name) :
    os.makedirs(output_directory+'/'+variant_name)
            
model_to_save = bert_model.module if hasattr(bert_model, "module") else bert_model
model_to_save.save_pretrained(output_directory+'/'+variant_name)
bert_tokenizer.save_pretrained(output_directory+'/'+variant_name)

## Prediction

predict on dev dataset

In [7]:
### Load two fine-tuned models, but with diffetent load methods

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_parameter_directory = [ f.path for f in os.scandir(output_directory) if f.is_dir() ]

# method = '_replace_v2'
method = '_append_v3'

# use catch file name
cache_file_name = 'bert-base-uncased_dev_with_T5{}'.format(method)
# use the predict file name
predict_file_name = 'coqa-dev-v1.0-{}_with_T5.json'.format(method[1:])
# # reset the catch file name
# cache_file_name = None
# # reset the predict file name
# predict_file_name = None

model_parameter_directory

['/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_rewritten_epoch4_append_v3.2',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_embedding_epoch2_batch2',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_embedding_epoch4_batch2',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_rewritten_epoch4_append_all',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/model_weights0']

In [9]:
t5_pooled = True

## load raw model
# t5 
t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_base_with_story_batch4_hist_20/epoch6.zip'
t5_model = torch.load(t5_path)
t5_input_tokenizer = t5base.get_input_tokenizer()   # or t5small 
t5_output_tokenizer = t5base.get_output_tokenizer()
t5_model.to(device)
t5_model.train(False)

# for m in model_parameter_directory:
m = model_parameter_directory[4]
variant_name = m.split('/')[-1]
print(variant_name)
# m = m + '/pytorch_model_2.bin'
bert_model = BertBaseUncasedModel_with_T5.from_pretrained(m) 
# bert_model = BertBaseUncasedModel.from_pretrained(m) 
bert_tokenizer = BertTokenizer.from_pretrained(m, do_lower_case=True)
bert_model.to(device)

## get datasets
t5_dataset = make_t5_dataset(input_dir,file_type='dev',t5_type='base',train_file=train_file)
cache_file_name = 'bert-base-uncased_dev_epoch2'
predict_file_name = 'coqa-dev-v1.0.json'

bert_dataset = load_dataset(bert_tokenizer, input_dir=input_dir, evaluate=True, cache_file_name=cache_file_name, predict_file_name=None, append_method='original')

if cache_file_name is not None:
    cache_file = os.path.join(input_dir,cache_file_name)

if os.path.exists(cache_file):
    features_and_dataset = torch.load(cache_file)
    features, dataset, examples = (
        features_and_dataset["features"],features_and_dataset["dataset"],features_and_dataset["examples"])

t5_dataset_new = []
for feature in features:
    t5_dataset_new.append(t5_dataset[feature.example_index])
t5_dataset = t5_dataset_new
del t5_dataset_new

# Write_predictions(model, tokenizer, device, variant_name, input_dir=input_dir, output_directory=output_directory, cache_file_name=cache_file_name, predict_file_name=predict_file_name, method=method, append_method='append')
Write_predictions_with_T5(bert_model, t5_model, bert_tokenizer, device, variant_name, t5_dataset, t5_pooled, input_dir=input_dir,output_directory=output_directory,cache_file_name=cache_file_name,predict_file_name=predict_file_name,evaluation_batch_size=1,method='', append_method='original')

model_weights0


Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


Creating features from dataset file at /content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data


Tag unique id to each example: 100%|██████████| 7983/7983 [00:00<00:00, 556188.91it/s]


Loading cache /content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/bert-base-uncased_dev_epoch2


Evaluating: 8565it [11:25, 12.49it/s]


save prediction file at: /content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/model_weights0/predictions.json


Writing preditions: 100%|██████████| 7983/7983 [00:30<00:00, 262.55it/s]


## Evaluation

In [10]:
evaluator = CoQAEvaluator(input_dir+'/'+predict_file_name)

# variant_name = 'Bert_with_T5_rewritten_epoch4_append_v2'

# variant_name = 'Bert_from_original_Surya_epoch4'

# variant_name = 'Bert_with_T5_rewritten_epoch4_replace'

method = '_pooled'

# m = model_parameter_directory[4]
# variant_name = m.split('/')[-1]

pre_file_bert = output_directory+'/'+variant_name+'/'+'predictions{}.json'.format(method)

# evaluate
with open(pre_file_bert) as f:
    pred_data = CoQAEvaluator.preds_to_dict(pre_file_bert)

# write evaluate results
with open(output_directory+'/'+variant_name+'/'+'evaluation{}.json'.format(method), 'w') as f:
    json.dump(evaluator.model_performance(pred_data), f, indent=2)

# show
print(json.dumps(evaluator.model_performance(pred_data), indent=2))

{
  "children_stories": {
    "em": 67.0,
    "f1": 76.2,
    "turns": 1425
  },
  "literature": {
    "em": 64.6,
    "f1": 74.2,
    "turns": 1630
  },
  "mid-high_school": {
    "em": 63.3,
    "f1": 73.8,
    "turns": 1653
  },
  "news": {
    "em": 67.8,
    "f1": 78.7,
    "turns": 1649
  },
  "wikipedia": {
    "em": 71.2,
    "f1": 80.2,
    "turns": 1626
  },
  "reddit": {
    "em": 0.0,
    "f1": 0.0,
    "turns": 0
  },
  "science": {
    "em": 0.0,
    "f1": 0.0,
    "turns": 0
  },
  "in_domain": {
    "em": 66.7,
    "f1": 76.6,
    "turns": 7983
  },
  "out_domain": {
    "em": 0.0,
    "f1": 0.0,
    "turns": 0
  },
  "overall": {
    "em": 66.7,
    "f1": 76.6,
    "turns": 7983
  }
}
